In [1]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("giphy.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10972 entries, 0 to 10971
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   GIF ID                10972 non-null  object
 1   Web Archive Link      10972 non-null  object
 2   MD5 Hash              10972 non-null  object
 3   File Size (In Bytes)  10972 non-null  int64 
 4   GIPHY Title           10461 non-null  object
 5   Import Date           10656 non-null  object
 6   Trending Date         10656 non-null  object
dtypes: int64(1), object(6)
memory usage: 600.2+ KB


In [3]:
giphy_data = df[
    df['GIPHY Title'].notnull() & 
    (df['GIPHY Title'] != '[empty]') & 
    (df['GIPHY Title'] != 'NA')
]
giphy_data

giphy_data['GIPHY Title'] = giphy_data['GIPHY Title'].str.replace(r'\bGIF\b', '', regex=True).str.strip()

/var/folders/0_/492vkfd907l731l24rh69fww0000gn/T/ipykernel_38993/3854612851.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  giphy_data['GIPHY Title'] = giphy_data['GIPHY Title'].str.replace(r'\bGIF\b', '', regex=True).str.strip()


In [5]:
import spacy
from textblob import TextBlob
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Define an enhanced preprocessing function
def preprocess_titles_enhanced(title):
    """
    Enhanced preprocessing for extracting sentiment and keywords from titles.
    """
    if pd.isna(title):
        return {"emotion": "unknown", "keywords": []}
    
    # Clean and tokenize the title using spaCy
    doc = nlp(title.lower())
    tokens = [token.lemma_ for token in doc if token.is_alpha and token.text not in ENGLISH_STOP_WORDS]
    
    # Extract sentiment polarity and categorize into emotions
    sentiment = TextBlob(title).sentiment.polarity
    if sentiment > 0.3:
        emotion = "positive"
    elif sentiment < -0.3:
        emotion = "negative"
    else:
        emotion = "neutral"
    
    # Extract keywords by filtering nouns and adjectives
    keywords = [token.text for token in doc if token.pos_ in {"NOUN", "ADJ"}]
    
    return {"emotion": emotion, "keywords": keywords}


giphy_data["processed"] = giphy_data["GIPHY Title"].apply(preprocess_titles_enhanced)

# Extract the processed information into separate columns for display
giphy_data["emotion"] = giphy_data["processed"].apply(lambda x: x["emotion"])
giphy_data["keywords"] = giphy_data["processed"].apply(lambda x: x["keywords"])

# Drop the intermediate 'Processed' column for clarity
dataset = giphy_data.drop(columns=['processed'])
dataset
# dataset.to_csv("cleangiphy.csv")

/var/folders/0_/492vkfd907l731l24rh69fww0000gn/T/ipykernel_38993/376279958.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  giphy_data["processed"] = giphy_data["GIPHY Title"].apply(preprocess_titles_enhanced)
/var/folders/0_/492vkfd907l731l24rh69fww0000gn/T/ipykernel_38993/376279958.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  giphy_data["emotion"] = giphy_data["processed"].apply(lambda x: x["emotion"])
/var/folders/0_/492vkfd907l731l24rh69fww0000gn/T/ipykernel_38993/376279958.py:39: SettingWit

,GIF ID,Web Archive Link,MD5 Hash,File Size (In Bytes),GIPHY Title,Import Date,Trending Date,emotion,keywords
0,1000eGIbYHercI,http://webarchive.loc.gov/all/20150318155641/h...,245bd1a346bf15da905a7acfd1985d87,82505,slipper,9/21/2013 3:37,1/1/1970 0:00,neutral,[slipper]
1,1000fHsBSKSL6w,http://webarchive.loc.gov/all/20150318155641/h...,a4328e5fcc99015f2cf04d533cadd06d,102564,swag hustling,6/22/2015 5:35,1/12/2016 23:45,neutral,[]
2,1000WjcUQeqOaY,http://webarchive.loc.gov/all/20150318155641/h...,1980e767b94acaf7978886989b755656,48315,jennifer lawrence salute,3/22/2013 11:27,1/1/1970 0:00,neutral,[]
3,1003Kuq13ICXDO,http://webarchive.loc.gov/all/20150318155641/h...,10b74b1cd9aaabbd259bc736d2be8db5,104921,love happy,3/7/2016 22:08,0000-00-00 00:00:00,positive,[happy]
4,1004D5v8Dbe6cw,http://webarchive.loc.gov/all/20150318155641/h...,ce3d7602047cb9125d42e098a418ece1,45649,visual album beyonce,12/13/2013 14:32,6/20/2014 7:15,neutral,"[visual, album, beyonce]"
...,...,...,...,...,...,...,...,...,...
10967,3o85xpNtPp5H1sLMbK,http://webarchive.loc.gov/all/20150318155641/h...,2ecddd73fbf21de12f521ced2370296a,73206,happy come home,4/30/2015 21:28,1/1/1970 0:00,positive,"[happy, home]"
10968,3o85xpOBewZNIN0QN2,http://webarchive.loc.gov/all/20150318155641/h...,88f6e072d409207e9ce0f5ba6d50f8a3,129256,dance love by Aron Shay,4/20/2015 8:36,0000-00-00 00:00:00,positive,"[dance, love]"
10969,3o85xpPcXtQmFu88Ra,http://webarchive.loc.gov/all/20150318155641/h...,80efbcc1fcccc1e906b79156e400c809,95407,day drawing by Doctor Popular,10/2/2015 19:07,0000-00-00 00:00:00,positive,"[day, doctor, popular]"
10970,3o85xpPS95XJLrwO6Q,http://webarchive.loc.gov/all/20150318155641/h...,9bb8dd22c71c0d77e8cbc676b44788f1,256369,pumpkin spice lol by Lance Ford,10/2/2015 19:43,0000-00-00 00:00:00,positive,"[spice, lol]"


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

# Prepare data for clustering
# Convert emotions to numerical labels
label_encoder = LabelEncoder()
emotion_labels = label_encoder.fit_transform(dataset['emotion'])

# Vectorize the keywords using TF-IDF
vectorizer = TfidfVectorizer(max_features=100, stop_words='english')
keywords_vectorized = vectorizer.fit_transform(dataset['keywords'].apply(lambda x: " ".join(x)))

# Combine emotion labels and TF-IDF vectors for clustering
from scipy.sparse import hstack
features = hstack([keywords_vectorized, emotion_labels.reshape(-1, 1)])

# Apply K-Means clustering
num_clusters = 5  # Set number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
dataset['Cluster'] = kmeans.fit_predict(features)

# Analyze cluster distribution
cluster_summary = dataset.groupby('Cluster').size()

cluster_summary


Cluster
0      82
1    8637
2     869
3     692
4     181
dtype: int64

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Function to process query
def process_query(query):
    # Use TextBlob for sentiment analysis to extract emotion from query
    sentiment = TextBlob(query).sentiment
    query_emotion = "positive" if sentiment.polarity > 0 else "negative" if sentiment.polarity < 0 else "neutral"
    
    # Vectorize the query keywords using the same vectorizer
    query_keywords_vector = vectorizer.transform([query])
    
    return query_emotion, query_keywords_vector

# Function to perform retrieval
def retrieve_gifs(query, top_n=10):
    # Process the query
    query_emotion, query_keywords_vector = process_query(query)
    
    # Match clusters by emotion similarity
    matching_clusters = dataset[dataset['emotion'] == query_emotion]
    
    if matching_clusters.empty:
        return "No matching clusters found for the specified emotion."

    # Compute cosine similarity for keywords within the matching clusters
    matching_keywords_vectors = vectorizer.transform(
        matching_clusters['keywords'].apply(lambda x: " ".join(x))
    )
    similarity_scores = cosine_similarity(query_keywords_vector, matching_keywords_vectors).flatten()
    
    # Add similarity scores to the dataset
    matching_clusters = matching_clusters.copy()
    matching_clusters['Similarity'] = similarity_scores
    
    # Rank by similarity score and select top N results
    top_results = matching_clusters.nlargest(top_n, 'Similarity')
    return top_results[['GIF ID', 'Web Archive Link', 'GIPHY Title', 'emotion', 'Similarity']]

# Example Query
query_happy_example = "happy celebration"
query_sad_example = "sad"
query_sports_celebration = "excited sports celebration"
happy_results = retrieve_gifs(query_happy_example)
sad_results = retrieve_gifs(query_sad_example)
sports_results = retrieve_gifs(query_sports_celebration)

sports_results

,GIF ID,Web Archive Link,GIPHY Title,emotion,Similarity
62,103JnbaqvpBFGE,http://webarchive.loc.gov/all/20150318155641/h...,excited tiger,positive,1.0
615,10pL8WrCh4ttXa,http://webarchive.loc.gov/all/20150318155641/h...,excited ally mcbeal,positive,1.0
972,113kTjzzweEzaE,http://webarchive.loc.gov/all/20150318155641/h...,excited rodney dangerfield,positive,1.0
1021,117hx0F5No4T0A,http://webarchive.loc.gov/all/20150318155641/h...,excited arrested development,positive,1.0
1308,11J3EBSwVYlaRa,http://webarchive.loc.gov/all/20150318155641/h...,excited rugrats,positive,1.0
1583,11sG4llQDAipGw,http://webarchive.loc.gov/all/20150318155641/h...,excited celebration,positive,1.0
2144,12inVGJ0hTaYY8,http://webarchive.loc.gov/all/20150318155641/h...,excited golden globes by Vulture.com,positive,1.0
2338,12PHrDCGTUoQIE,http://webarchive.loc.gov/all/20150318155641/h...,excited rodney dangerfield,positive,1.0
2509,12wKtJWXDVpRao,http://webarchive.loc.gov/all/20150318155641/h...,excited glitter by Sony Music Perú,positive,1.0
3810,14Xx5B178hibS,http://webarchive.loc.gov/all/20150318155641/h...,excited ian somerhalder,positive,1.0
